In [1]:
%pwd

'c:\\Users\\timil\\OneDrive\\Documents\\My_ML project\\End-to-end-chicken-Disease-Classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\timil\\OneDrive\\Documents\\My_ML project\\End-to-end-chicken-Disease-Classification'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_path: Path
    unzip_dir: Path

In [12]:
from src.Chicken_Disease_Classification.constant import *
from src.Chicken_Disease_Classification.utils.common import read_yaml, create_directories 

In [13]:

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        #self.params = read_yaml(params_filepath)

        create_directories([Path(self.config["artifacts_root"])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]

        # Create the directory where the ZIP will be downloaded and extracted
        create_directories([
            Path(config["local_data_path"]).parent,
            Path(config["unzip_dir"])
        ])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(self.config["artifacts_root"]),
            source_URL=config["source_URL"],
            local_data_path=Path(config["local_data_path"]),
            unzip_dir=Path(config["unzip_dir"])
        )
        return data_ingestion_config

In [14]:
import os
import urllib.request as request 
from src.Chicken_Disease_Classification.utils.logger import logger
from src.Chicken_Disease_Classification.utils.common import get_size

In [23]:
import os
from pathlib import Path
import py7zr
from urllib import request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_path):
            filename, _ = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_path
            )
            logger.info(f" Downloaded file: {filename} | Size: {get_size(Path(filename))}")
        else:
            logger.info(f" File already exists at: {self.config.local_data_path}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with py7zr.SevenZipFile(self.config.local_data_path, mode='r') as archive:
            archive.extractall(path=unzip_path)

        logger.info(f" Extracted 7z archive to: {unzip_path}")

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error during data ingestion: {e}")
    raise e 

[2025-08-22 03:53:02,738: INFO: common: Directory created at: artifacts]
[2025-08-22 03:53:02,740: INFO: common: Directory created at: artifacts\data_ingestion]
[2025-08-22 03:53:02,742: INFO: common: Directory created at: artifacts\data_ingestion\unzipped]
[2025-08-22 03:53:02,744: INFO: 388049438:  File already exists at: artifacts\data_ingestion\data.zip]
[2025-08-22 03:53:03,697: INFO: 388049438:  Extracted 7z archive to: artifacts\data_ingestion\unzipped]
